## **Mapeamento Galp**

##### **___Informações___**

**Projeto:** Galp

**Autor:** Lucas Pimenta dos Santos Monteiro

**Data:** 24/02/2025

##### **___Objetivo___**

A ideia desse cógido é automatizar a visualização de uma imagem .svg gerada no Inkscape de acordo com o output do programa (report.csv) e os dados de entrada definidos (entry_data.csv).

Para isso foram criados rótulos no Inkscape que identificam cada elemento de texto a partir do inkscape:label. Cada label recebeu o nome equivalente a variável proveniente do report.csv e entry_data.csv que representa o valor desejado.

##### **Importando as Bibliotecas**

In [405]:
import pandas as pd
import xml.etree.ElementTree as ET

In [406]:
entry_data = pd.read_csv("entry_data.csv")
report = pd.read_csv("report.csv")

##### **Verificando os Dados**

In [407]:
entry_data.head()

,Entry data:,Unnamed: 1,Unnamed: 2
0,Entry feed mass flow rate =,600.0000000000,kg/h
1,Vacuum pressure =,-495.8300000000,mbar
2,Membrane area =,12.9600000000,m²
3,Membrane thickness =,100.0000000000,microns
4,Membrane porosity =,85.0000000000%,NaN


In [408]:
report.head()

,Time (s),Desal - Coolant mass flow rate (kg/h),Desal - Hot feedwater inlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Cold feedwater inlet temperature (°C),Desal - Cold feedwater outlet temperature (°C),Desal - Feed membrane temperature (°C),Desal - Membrane gap temperature (°C),Desal - Gap film boundary temperature (°C),Desal - Film wall temperature (°C),...,ETC - Solar irradiance (W/m²),PVT - Mass flow rate (kg/s),PVT - PVT inlet temperature (°C),PVT - PVT outlet temperature (°C),PVT - Thermal Efficiency,PVT - Heat transfer rate (W),PVT - Electrical Efficiency,PVT - Electrical power output (W),PVT - Electrical energy generated (kWh),PVT - Solar irradiance (W/m²)
0,126.0,600.0,52.188594,34.215036,30.172547,48.144390,42.822386,41.830193,40.774333,39.773411,...,601.841893,0.2266,52.257177,53.299641,43.646903,987.696913,15.274955,345.660849,0.012015,601.841893
1,252.0,600.0,52.042543,34.179068,30.154804,48.016568,42.733295,41.744399,40.692131,39.697218,...,609.842381,0.2266,52.110802,53.178955,44.135574,1012.031953,15.280602,350.385321,0.024197,609.842381
2,378.0,600.0,51.921259,34.148602,30.139347,47.910294,42.659001,41.672810,40.623489,39.633540,...,617.750464,0.2266,51.989249,53.082279,44.585175,1035.598465,15.284828,355.027076,0.036542,617.750464
3,504.0,600.0,51.823202,34.123195,30.125879,47.824179,42.598531,41.614472,40.567476,39.581501,...,625.566142,0.2266,51.890975,53.008107,44.998813,1058.430002,15.287723,359.586924,0.049049,625.566142
4,630.0,600.0,51.746913,34.102440,30.114146,47.756914,42.550968,41.568490,40.523220,39.540290,...,633.289414,0.2266,51.814520,52.955010,45.379335,1080.558290,15.289373,364.065699,0.061714,633.289414


##### **Definindo as Colunas que serão utilizadas de cada dataframe**

In [409]:
userows_entry_data = [
    "Cooling heat exchanger flow rate =",
    "Brine flow rate out of the regulating tank =",
    "Brackish water temperature at the entrance of the regulating tank =",
    "Brackish water salinity at the entrance of the regulating tank =",
    "Temperature at the entrance of the cold side =",
    "Heating heat exchanger mass flow rate ="
]

usecols_report = [
"Time (s)",
"Desal - Total water produced (kg)",
"Desal - Distilled water production rate (kg/h)",
"FeedTank - Brackish water inlet flow rate (kg/h)",
"ColdHex - Heat transfer rate (W)",
"ColdHex - Cold side outlet temperature (°C)",
"Desal - Hot feedwater outlet temperature (°C)",
"Desal - Hot feedwater outlet mass flow rate (kg/h)",
"Desal - Hot feedwater outlet salinity (wt%)",
"HotHex - Cold side salinity (wt%)",
"ColdHex - Hot side outlet temperature (°C)",
"ColdHex - Hot side salinity (wt%)",
"HotHex - Cold side salinity (wt%)",
"HotHex - Heat transfer rate (W)",
"HotHex - Hot side outlet temperature (°C)",
"HotHex - Hot side inlet temperature (°C)",
"HotHex - Cold side outlet temperature (°C)",
"ColdHex - Hot side inlet temperature (°C)",
"ColdHex - Hot side inlet mass flow rate (kg/h)",
"HotHex - Cold side inlet temperature (°C)",
"Time (s)",
"SolarField - Solar irradiance (W/m²)", # segundo 11088 -> tempo de máximo
"SolarField - Efficiency",
"SolarField - Solar field inlet temperature (°C)",
"SolarField - Solar field outlet temperature (°C)",
"CircuitFromSolar - Outlet temperature (°C)",
"CircuitToSolar - Inlet temperature (°C)",
"ColdHex - Hot side salinity (wt%)",
"FeedTank - Water salinity (wt%)",
"FeedTank - Water temperature (°C)",
"Desal - Film wall temperature (°C)",
"Desal - Gap film boundary temperature (°C)",
"HCPV - Inlet temperature (°C)",
"HCPV - Outlet temperature (°C)",
"HCPV - Mass flow rate (kg/s)",
"HCPV - Heat transfer rate recovered from the solar cells (kW)",
"HCPV - Electrical power (kW)",
"HCPV - Global panel electrical efficiency (%)",
"HCPV - Electrical energy generated (kWh)",
"CircuitToSolar - Mass flow rate (kg/s)",
"ETC - Efficiency",
"ETC - ETC inlet temperature (°C)",
"ETC - ETC outlet temperature (°C)",
"ETC - Mass flow rate (kg/s)",
"PVT - PVT inlet temperature (°C)",
"PVT - Thermal Efficiency",
"PVT - Electrical power output (W)",
"PVT - Electrical Efficiency",
"PVT - Electrical energy generated (kWh)",
"PVT - PVT outlet temperature (°C)",
"PVT - Mass flow rate (kg/s)",
"PVT - Heat transfer rate (W)",
"ETC - Heat transfer rate (W)",
"SolarField - Heat transfer rate (W)",
"Reservoir - Temperature (°C)"
]

##### Filtrando os Dataframes de Acordo com as Colunas

In [410]:
entry_data_filtrado = entry_data[entry_data['Entry data:'].isin(userows_entry_data)]
entry_data_filtrado.head()

,Entry data:,Unnamed: 1,Unnamed: 2
19,Heating heat exchanger mass flow rate =,0.3888888890,kg/s
20,Cooling heat exchanger flow rate =,2000.0000000000,kg/h
21,Temperature at the entrance of the cold side =,30.0000000000,°C
24,Brackish water salinity at the entrance of the...,1.0000000000,wt%
25,Brackish water temperature at the entrance of ...,25.0000000000,°C


In [411]:
report_filtrado = report.filter(items=usecols_report)
report_filtrado.head()

,Time (s),Desal - Total water produced (kg),Desal - Distilled water production rate (kg/h),FeedTank - Brackish water inlet flow rate (kg/h),ColdHex - Heat transfer rate (W),ColdHex - Cold side outlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Hot feedwater outlet mass flow rate (kg/h),Desal - Hot feedwater outlet salinity (wt%),HotHex - Cold side salinity (wt%),...,PVT - Thermal Efficiency,PVT - Electrical power output (W),PVT - Electrical Efficiency,PVT - Electrical energy generated (kWh),PVT - PVT outlet temperature (°C),PVT - Mass flow rate (kg/s),PVT - Heat transfer rate (W),ETC - Heat transfer rate (W),SolarField - Heat transfer rate (W),Reservoir - Temperature (°C)
0,126.0,0.420438,11.959442,11.959442,12787.926527,35.502560,34.215036,588.040558,1.023198,1.002803,...,43.646903,345.660849,15.274955,0.012015,53.299641,0.2266,987.696913,1603.069531,3023.501620,52.257177
1,252.0,0.837333,11.866633,11.866633,11475.182015,34.937581,34.179068,588.133367,1.025872,1.005582,...,44.135574,350.385321,15.280602,0.024197,53.178955,0.2266,1012.031953,1632.051807,3116.310273,52.110802
2,378.0,1.251267,11.789919,11.789919,10331.127175,34.445222,34.148602,588.210081,1.028553,1.008342,...,44.585175,355.027076,15.284828,0.036542,53.082279,0.2266,1035.598465,1660.321711,3205.645205,51.989249
3,504.0,1.662784,11.728283,11.728283,9334.154685,34.016178,34.123195,588.271717,1.031243,1.011085,...,44.998813,359.586924,15.287723,0.049049,53.008107,0.2266,1058.430002,1687.905174,3291.666751,51.890975
4,630.0,2.072396,11.680770,11.680770,8465.449936,33.642345,34.102440,588.319230,1.033945,1.013816,...,45.379335,364.065699,15.289373,0.061714,52.955010,0.2266,1080.558290,1714.826566,3374.526755,51.814520


##### Separando apenas o tempo final do output gerado no report.csv

In [412]:
report_filtrado.tail()

,Time (s),Desal - Total water produced (kg),Desal - Distilled water production rate (kg/h),FeedTank - Brackish water inlet flow rate (kg/h),ColdHex - Heat transfer rate (W),ColdHex - Cold side outlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Hot feedwater outlet mass flow rate (kg/h),Desal - Hot feedwater outlet salinity (wt%),HotHex - Cold side salinity (wt%),...,PVT - Thermal Efficiency,PVT - Electrical power output (W),PVT - Electrical Efficiency,PVT - Electrical energy generated (kWh),PVT - PVT outlet temperature (°C),PVT - Mass flow rate (kg/s),PVT - Heat transfer rate (W),ETC - Heat transfer rate (W),SolarField - Heat transfer rate (W),Reservoir - Temperature (°C)
195,24696.0,107.643255,8.934401,8.934401,2450.327664,31.054156,33.442520,591.065599,1.743585,1.717622,...,37.444936,242.906447,15.601118,3.063026,48.478135,0.2266,583.010559,1049.027283,1657.933784,47.862739
196,24822.0,107.952406,8.732829,8.732829,2420.952547,31.041517,33.389945,591.267171,1.745082,1.719683,...,37.083909,237.371691,15.623637,3.071430,48.135322,0.2266,563.420033,1020.768378,1594.124636,47.540602
197,24948.0,108.254486,8.530380,8.530380,2391.044534,31.028650,33.336514,591.469620,1.746527,1.721697,...,36.702779,231.769064,15.646390,3.079639,47.788896,0.2266,543.676116,992.243063,1529.771992,47.215017
198,25074.0,108.549466,8.327100,8.327100,2360.596502,31.015551,33.282217,591.672900,1.747922,1.723663,...,36.299711,226.098060,15.669376,3.087651,47.438862,0.2266,523.779255,963.450877,1464.875454,46.885985
199,25200.0,108.837317,8.123033,8.123033,2329.601209,31.002215,33.227043,591.876967,1.749264,1.725582,...,35.872642,220.358168,15.692596,3.095464,47.085223,0.2266,503.729903,934.391356,1399.434633,46.553508


In [413]:
report_filtrado = report_filtrado.iloc[[report_filtrado['SolarField - Solar irradiance (W/m²)'].idxmax()]].reset_index(drop=True)
report_filtrado.head()

,Time (s),Desal - Total water produced (kg),Desal - Distilled water production rate (kg/h),FeedTank - Brackish water inlet flow rate (kg/h),ColdHex - Heat transfer rate (W),ColdHex - Cold side outlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Hot feedwater outlet mass flow rate (kg/h),Desal - Hot feedwater outlet salinity (wt%),HotHex - Cold side salinity (wt%),...,PVT - Thermal Efficiency,PVT - Electrical power output (W),PVT - Electrical Efficiency,PVT - Electrical energy generated (kWh),PVT - PVT outlet temperature (°C),PVT - Mass flow rate (kg/s),PVT - Heat transfer rate (W),ETC - Heat transfer rate (W),SolarField - Heat transfer rate (W),Reservoir - Temperature (°C)
0,11088.0,46.11811,18.985883,18.985883,3441.599221,31.48064,35.463875,581.014117,1.350178,1.307454,...,48.204124,521.983284,14.579491,1.438155,63.728029,0.2266,1725.831705,2605.039634,5418.374974,61.907649


##### Lendo o arquivo do Inkscape e definindo uma função de substituição de valores

In [414]:
# Caminho para o arquivo SVG
arquivo_svg = 'Diag_geral_24.02.2025.svg'

# Parse o arquivo SVG
tree = ET.parse(arquivo_svg)
root = tree.getroot()

# Definir namespaces
namespaces = {'svg': 'http://www.w3.org/2000/svg', 'inkscape': 'http://www.inkscape.org/namespaces/inkscape'}

def substituir_texto_por_label(label, novo_texto):
    # Encontre todos os elementos do tipo 'text'
    textos = root.findall('.//svg:text', namespaces)
    
    # Para cada elemento 'text' encontrado
    for texto in textos:
        # Verifique se o elemento 'text' possui o atributo 'inkscape:label'
        if texto.attrib.get('{http://www.inkscape.org/namespaces/inkscape}label') == label:
            # Itere sobre os elementos 'tspan' dentro do elemento 'text'
            for tspan in texto.findall('.//svg:tspan', namespaces):
                # Substitua o texto do 'tspan' pelo novo texto
                tspan.text = novo_texto

#### Definindo Função para Cálculo da Salinidade em g/l

Atualmente o código gera no report a salinidade em porcentagem de peso wt% (weight percent), porém a salinidade depende da densidade que é inversamente proporcional a temperatura, por isso é necessário o uso da função a seguir:

In [415]:
def salt_water_density(T, s):
    a0 = 9.999 * 10**2
    a1 = 2.034 * 10**-2
    a2 = -6.162 * 10**-3
    a3 = 2.261 * 10**-5
    a4 = -4.657 * 10**-8
    b0 = 8.020 * 10**2
    b1 = -2.001
    b2 = 1.677 * 10**-2
    b3 = -3.060 * 10**-5
    b4 = -1.613 * 10**-5

    result = (a0 + a1 * T + a2 * T**2 + a3 * T**3 + a4 * T**4 + 
              b0 * s + b1 * s * T + b2 * s * T**2 + 
              b3 * s * T**3 + b4 * s**2 * T**2)

    return result

#### Definindo cada a temperatura para cada Salinidade

In [416]:
# De cima para baixo no desenho e da esquerda para a direita, temos

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### HotHex - Cold side salinity (wt%) -> está presente na primeira e na segunda instância de salinidade, os valores são iguais em wt% mas distintos em g/L


# A temperatura da primeira instância (mais a esquerda) é a variável HotHex - Cold side outlet temperature (°C)

# No desenho, o nome dessa instância da salinidade foi alterado para HotHex - Cold side salinity 1 (wt%)

T_css1 = float(report_filtrado['HotHex - Cold side outlet temperature (°C)'].iloc[0])
S_css1 = float(report_filtrado['HotHex - Cold side salinity (wt%)'].iloc[0]/100)
css1_g_l = "s " + str(round(S_css1 * salt_water_density(T_css1, S_css1), 2)) + " g/l"

# A temperatura da segunda instância (mais a direita) é a variável HotHex - Cold side inlet temperature (°C)
# No desenho, o nome dessa instância da salinidade foi alterado para HotHex - Cold side salinity 2 (wt%)

T_css2 = float(report_filtrado['HotHex - Cold side inlet temperature (°C)'].iloc[0])
S_css2 = float(report_filtrado['HotHex - Cold side salinity (wt%)'].iloc[0]/100)
css2_g_l = "s " + str(round(S_css2 * salt_water_density(T_css2, S_css2), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### Desal - Hot feedwater outlet salinity (wt%) -> terceira instância de salinidade

# A temperatura da terceira instância está na variável -> Desal - Hot feedwater outlet temperature (°C)

T_hfos = float(report_filtrado['Desal - Hot feedwater outlet temperature (°C)'].iloc[0])
S_hfos = float(report_filtrado['Desal - Hot feedwater outlet salinity (wt%)'].iloc[0]/100)
hfos_g_l = "s " + str(round(S_hfos * salt_water_density(T_hfos, S_hfos), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### ColdHex - Hot side salinity (wt%) -> está presente na quarta e quinta instância de salinidade

# A temperatura da quarta instância (mais para cima) é a variável ColdHex - Hot side outlet temperature (°C)
# No desenho, o nome dessa instância da salinidade foi alterado para ColdHex - Hot side salinity 1 (wt%)

T_hss1 = float(report_filtrado['ColdHex - Hot side outlet temperature (°C)'].iloc[0])
S_hss1 = float(report_filtrado['ColdHex - Hot side salinity (wt%)'].iloc[0]/100)
hss1_g_l = "s " + str(round(S_hss1 * salt_water_density(T_hss1, S_hss1), 2)) + " g/l"

# A temperatura da quinta instância (mais para baixo) é a variável ColdHex - Hot side inlet temperature (°C)
# No desenho, o nome dessa instância da salinidade foi alterado para ColdHex - Hot side salinity 2 (wt%)

T_hss2 = float(report_filtrado['ColdHex - Hot side inlet temperature (°C)'].iloc[0])
S_hss2 = float(report_filtrado['ColdHex - Hot side salinity (wt%)'].iloc[0]/100)
hss2_g_l = "s " + str(round(S_hss2 * salt_water_density(T_hss2, S_hss2), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### Brackish water salinity at the entrance of the regulating tank = -> está presente na sexta instância de salinidade

# A temperatura da sexta instância está na variável -> Brackish water temperature at the entrance of the regulating tank =

T_bws = float(entry_data_filtrado[entry_data_filtrado['Entry data:'] == 'Brackish water temperature at the entrance of the regulating tank =']['Unnamed: 1'].iloc[0])
S_bws = float(entry_data_filtrado[entry_data_filtrado['Entry data:'] == 'Brackish water salinity at the entrance of the regulating tank =']['Unnamed: 1'].iloc[0])/100
bws_g_l = "s " + str(round(S_bws * salt_water_density(T_bws, S_bws), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### FeedTank - Water salinity (wt%) -> está presente na sétima instância de salinidade

# A temperatura da sétima instância está na variável -> FeedTank - Water temperature (°C)

T_fts = float(report_filtrado['FeedTank - Water temperature (°C)'].iloc[0])
S_fts = float(report_filtrado['FeedTank - Water salinity (wt%)'].iloc[0]/100)
fts_g_l = "s " + str(round(S_fts * salt_water_density(T_fts, S_fts), 2)) + " g/l"

#### Passando o Desal - Total water produced (kg) para litro -> vamos dividir o valor em kg por 0.998

In [417]:
total_water_produced_kg = report_filtrado['Desal - Total water produced (kg)'].iloc[0]
total_water_produced_l = str(round(total_water_produced_kg / 0.998, 2)) + " L"
print(total_water_produced_l)

46.21 L


#### Calculando o valor de (Desal - Film wall temperature (°C)+Desal - Gap film boundary temperature (°C))/2

In [418]:
film_wall_temp = report_filtrado['Desal - Film wall temperature (°C)'].iloc[0]
gap_film_boundary_temp = report_filtrado['Desal - Gap film boundary temperature (°C)'].iloc[0]
average_temp = str(round((film_wall_temp + gap_film_boundary_temp) / 2, 2)) + " °C"

##### Substituindo os Valores

In [419]:
for coluna, valor in report_filtrado.items():
    valor = valor.iloc[0]
    grandeza = ""
    if '(' in coluna and ')' in coluna:
        unidade = coluna[coluna.rfind('(') + 1:coluna.rfind(')')]
        unidade = unidade.strip()
    elif coluna == "SolarField - Efficiency" or coluna == "PVT - Thermal Efficiency" or coluna == "ETC - Efficiency" or coluna == "PVT - Electrical Efficiency":
        unidade = "%"
    else:
        unidade = ""
    if unidade == '°C':
        grandeza = "T"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'kg/s':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'Kg/h' or unidade == 'kg/h':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'g/L':
        grandeza = "S"
        valor = "{:.2f}".format(float(valor))
    valor = "{:.2f}".format(float(valor))
    valor_novo = grandeza + " " + str(valor) + " " + unidade
    print(valor_novo)
    substituir_texto_por_label(coluna, valor_novo)

 11088.00 s
 46.12 kg
ṁ 18.99 kg/h
ṁ 18.99 kg/h
 3441.60 W
T 31.48 °C
T 35.46 °C
ṁ 581.01 kg/h
 1.35 wt%
 1.31 wt%
T 30.05 °C
 1.31 wt%
 3720.05 W
T 59.62 °C
T 61.91 °C
T 61.82 °C
T 35.06 °C
ṁ 600.00 kg/h
T 56.40 °C
 952.20 W/m²
 47.42 %
T 61.80 °C
T 65.61 °C
T 65.54 °C
T 61.91 °C
 1.31 wt%
T 35.06 °C
T 44.09 °C
T 45.54 °C
T 61.91 °C
T 58.55 °C
ṁ 0.34 kg/s
 -4.77 kW
 4.13 kW
 24.11 %
 11.27 kWh
ṁ 0.34 kg/s
 61.34 %
T 61.91 °C
T 64.66 °C
ṁ 0.23 kg/s
T 61.91 °C
 48.20 %
 521.98 W
 14.58 %
 1.44 kWh
T 63.73 °C
ṁ 0.23 kg/s
 1725.83 W
 2605.04 W
 5418.37 W
T 61.91 °C


In [420]:
for linha in entry_data_filtrado.itertuples():
    grandeza = ""
    label = linha[1].rstrip(' ')
    valor = linha[2]
    valor = "{:.2f}".format(float(valor))
    unidade = linha[3].strip()
    if unidade == '°C':
        grandeza = "T"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'kg/s':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'kg/h' or unidade == 'Kg/h':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'g/L':
        grandeza = "S"
        valor = "{:.2f}".format(float(valor))

    valor_novo = grandeza + " " + str(valor) + ' ' + unidade
    #print(valor_novo)
    substituir_texto_por_label(label, str(valor_novo))

In [421]:
substituir_texto_por_label("HotHex - Cold side salinity 1 (wt%)", css1_g_l)
substituir_texto_por_label("HotHex - Cold side salinity 2 (wt%)", css2_g_l)
substituir_texto_por_label("Desal - Hot feedwater outlet salinity (wt%)", hfos_g_l)
substituir_texto_por_label("ColdHex - Hot side salinity 1 (wt%)", hss1_g_l)
substituir_texto_por_label("ColdHex - Hot side salinity 2 (wt%)", hss2_g_l)
substituir_texto_por_label("Brackish water salinity at the entrance of the regulating tank =", bws_g_l)
substituir_texto_por_label("FeedTank - Water salinity (wt%)", fts_g_l)

# correção do valor de total de água produzida
substituir_texto_por_label("Desal - Total water produced (kg)", total_water_produced_l)

# correção do valor de temperatura média
substituir_texto_por_label("(Desal - Film wall temperature (°C)+Desal - Gap film boundary temperature (°C))/2", average_temp)

##### Salvando o arquivo

In [422]:
# Salvar as alterações no arquivo SVG
tree.write('Diag_geral_modificado.svg')